In [14]:
!pip install -q transformers sentencepiece

In [15]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

In [17]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_source_code_summarization_python"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_source_code_summarization_python", skip_special_tokens=True),
    device="mps"
)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [18]:
code = '''with open("file.txt", "r") as in_file:\n    buf = in_file.readlines()\n\nwith open("file.txt", "w") as out_file:\n    for line in buf:\n        if line == "; Include this text\n":\n            line = line + "Include below\n"\n        out_file.write(line)''' #@param {type:"raw"}

In [19]:
import tokenize
import io

def pythonTokenizer(line):
    result= []
    line = io.StringIO(line) 
    
    for toktype, tok, start, end, line in tokenize.generate_tokens(line.readline):
        if (not toktype == tokenize.COMMENT):
            if toktype == tokenize.STRING:
                result.append("CODE_STRING")
            elif toktype == tokenize.NUMBER:
                result.append("CODE_INTEGER")
            elif (not tok=="\n") and (not tok=="    "):
                result.append(str(tok))
    return ' '.join(result)

In [20]:
tokenized_code = pythonTokenizer(code)
print("code after tokenization " + tokenized_code)

code after tokenization with open ( CODE_STRING , CODE_STRING ) as in_file : buf = in_file . readlines ( )  with open ( CODE_STRING , CODE_STRING ) as out_file : for line in buf :          if line ==   " ; Include this text   " :              line = line +   " Include below  "          out_file . write ( line )   


In [21]:
pipeline([tokenized_code])

/opt/homebrew/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:2578: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  if unfinished_sequences.max() == 0:


[{'summary_text': 'How to read a dbf tables into a single record ?'}]